In [26]:
#..........................案例1:
#..........................K-means
import numpy as np
import pandas as pd
# 欧氏距离
def distEclud(vecA, vecB):
    return np.sqrt(np.sum(np.power(vecA - vecB, 2))) #la.norm(vecA-vecB)
# k个质心，每个维度随机选择k个范围内的点
def randCent(dataSet, k):
    n = np.shape(dataSet)[1]
    centroids = np.mat(np.zeros((k,n)))#create centroid mat
    for j in range(n):#create random cluster centers, within bounds of each dimension
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = np.mat(minJ + rangeJ * np.random.rand(k,1))
    return centroids
file=r'C:\Users\Wudey\Desktop\machinelearninginaction\Ch10\testSet_KM.txt'
dta=pd.read_table(file,header=None)
print('K-means的k个点：\n',randCent(np.mat(dta),3))
# 计算每个点与k个点的距离，这个点归属于最近的K
# 分配完以后，重新计算k的值（均值）
# 再计算距离，如此反复
def kMeans(dataSet, k, distMeas=distEclud, createCent=randCent):
    m = np.shape(dataSet)[0]
    clusterAssment = np.mat(np.zeros((m,2))) # 第一列是标记第几个K，第一个是每个点的SE
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m): # for each data point assign it to the closest centroid
            minDist = np.inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        print(centroids)
        for cent in range(k): # recalculate centroids
            ptsInClust = dataSet[np.nonzero(clusterAssment[:,0].A==cent)[0]] # get all the point in this cluster
            centroids[cent,:] = np.mean(ptsInClust, axis=0) #assign centroid to mean
    return centroids, clusterAssment
print('迭代过程(迭代了3次)：')
np.random.seed(0)
K=kMeans(np.mat(dta),4)

K-means的k个点：
 [[ 4.4668495   0.75118342]
 [-1.46176469  1.12008995]
 [ 2.71001547  4.48929816]]
迭代过程(迭代了3次)：
[[ 0.22798161 -0.24049276]
 [ 1.92798538  1.85366518]
 [ 0.77923336 -0.10920783]
 [ 0.18782218  4.1705785 ]]
[[-3.02774732 -2.48702036]
 [ 2.97214218  2.22480429]
 [ 2.84110669 -2.83317019]
 [-1.29906809  3.36493886]]
[[-3.38237045 -2.9473363 ]
 [ 2.6265299   3.10868015]
 [ 2.80293085 -2.7315146 ]
 [-2.46154315  2.78737555]]


In [30]:
#..........................案例2:
#..........................聚类效果会受到初始值的影响
#............有可能聚类到局部最优。
#... 用SSE来看聚类效果，一种调整策略是将方差最大的进行划分（提取出来跑K-means），然后将临近的K合并（一种合并方法是直接算距离，合并最近的，另一种是合并SSE增加最小的两个K）。
#... 二分K-means
# 所有点作为1簇，然后划分为2，再选择其中一个继续划分，使得整体SSE下降最大，循环往复直到K
def biKmeans(dataSet, k, distMeas=distEclud):
    m = np.shape(dataSet)[0]
    clusterAssment = np.mat(np.zeros((m,2)))
    centroid0 = np.mean(dataSet, axis=0).tolist()[0]
    centList =[centroid0] #create a list with one centroid
    for j in range(m):#calc initial Error
        clusterAssment[j,1] = distMeas(np.mat(centroid0), dataSet[j,:])**2
    while (len(centList) < k):
        lowestSSE = np.inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[np.nonzero(clusterAssment[:,0].A==i)[0],:]#get the data points currently in cluster i
            centroidMat, splitClustAss = kMeans(ptsInCurrCluster, 2, distMeas)
            sseSplit = sum(splitClustAss[:,1])#compare the SSE to the currrent minimum
            sseNotSplit = sum(clusterAssment[np.nonzero(clusterAssment[:,0].A!=i)[0],1])
            print("sseSplit, and notSplit: ",sseSplit,sseNotSplit)
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[np.nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList) #change 1 to 3,4, or whatever
        bestClustAss[np.nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        print('the bestCentToSplit is: ',bestCentToSplit)
        print('the len of bestClustAss is: ', len(bestClustAss))
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]#replace a centroid with two best centroids
        centList.append(bestNewCents[1,:].tolist()[0])
        clusterAssment[np.nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:]= bestClustAss#reassign new clusters, and SSE
    return np.mat(centList), clusterAssment
print('二分K-means：')
c,cl=biKmeans(np.mat(dta),3)
print('质心：\n',c)





二分K-means：
[[ 3.18228321  4.96857787]
 [-4.39779379  0.18350771]]
[[ 2.81771379  1.8327775 ]
 [-1.67663544 -0.90333988]]
[[ 2.82324713  1.35829639]
 [-1.95530037 -0.77067535]]
[[ 2.80649627  0.59102776]
 [-2.60766091 -0.40753328]]
[[ 2.77278464  0.27809051]
 [-2.83969898 -0.15857156]]
[[ 2.71473038  0.18858278]
 [-2.9219568  -0.07998038]]
sseSplit, and notSplit:  [[828.692654]] 0
the bestCentToSplit is:  0
the len of bestClustAss is:  80
[[ 4.73617672  2.78798427]
 [ 3.10438823 -3.66248813]]
[[ 2.6265299   3.10868015]
 [ 2.80293085 -2.7315146 ]]
sseSplit, and notSplit:  [[87.30471265]] [[399.99802122]]
[[-3.96925767 -1.72320443]
 [-4.78025152 -3.2265316 ]]
[[-2.66266977  0.89528016]
 [-3.81505656 -3.43921111]]
[[-2.46154315  2.78737555]
 [-3.38237045 -2.9473363 ]]
sseSplit, and notSplit:  [[62.64959203]] [[428.69463278]]
the bestCentToSplit is:  0
the len of bestClustAss is:  40
质心：
 [[ 2.6265299   3.10868015]
 [-2.9219568  -0.07998038]
 [ 2.80293085 -2.7315146 ]]
